## Day 01

In [1]:
nums = parse.(Int, readlines("01.txt"))
@assert nums[end] == 10044

In [2]:
function window_sum(idx, window_size)
    sum(nums[idx - window_size + 1:idx])
end

function count_increasing(window_size)
    result = 0
    for idx in window_size + 1:length(nums)
        if window_sum(idx, window_size) > window_sum(idx - 1, window_size)
            result += 1
        end
    end
    result
end

count_increasing (generic function with 1 method)

In [3]:
println("easy: ", count_increasing(1))
println("hard: ", count_increasing(3))

easy: 1715
hard: 1739


## Day 02

In [4]:
lines = readlines("02.txt")

function move_submarine(is_hard)
    Δx, Δy = 0, 0
    aim = 0

    for line in lines
        action, arg = split(line)
        arg = parse(Int, arg)

        if action == "forward"
            Δx += arg
            Δy += aim * arg
        else
            inc = action == "down" ? arg : -arg
            if is_hard
                aim += inc
            else
                Δy += inc
            end
        end
    end
    
    Δx * Δy
end

move_submarine (generic function with 1 method)

In [5]:
println("easy: ", move_submarine(false))
println("hard: ", move_submarine(true))

easy: 1714950
hard: 1281977850


## Day 03

`mcb` stands for Most Common Bit. 

In [6]:
lines = readlines("03.txt")
n_bits = length(lines[1])
all_nums = reduce(vcat, permutedims.(collect.(lines)))

function get_mcb(nums, bit_idx, should_invert)
    n_ones = count(==('1'), nums[:, bit_idx])
    ones_are_mcb = 2 * n_ones >= size(nums, 1)
    (ones_are_mcb ⊻ should_invert) ? '1' : '0'
end

from_chars(x) = parse(Int, join(x), base=2)

function day03_easy()
    function get_rate(should_invert)
        from_chars([
            get_mcb(all_nums, bit_idx, should_invert)
            for bit_idx in 1:n_bits
        ])
    end

    γ_rate = get_rate(false)
    ϵ_rate = get_rate(true)
    γ_rate * ϵ_rate
end

function day03_hard()
    function get_rating(should_invert)
        cur_nums = all_nums
        bit_idx = 1

        while size(cur_nums, 1) > 1
            needed_bit = get_mcb(cur_nums, bit_idx, should_invert)
            cur_nums = cur_nums[cur_nums[:, bit_idx] .== needed_bit, :]
            bit_idx += 1
        end
        
        from_chars(cur_nums)
    end
    
    o₂_rating = get_rating(false)
    co₂_rating = get_rating(true)
    o₂_rating * co₂_rating
end

println("easy: ", day03_easy())
println("hard: ", day03_hard())

easy: 2640986
hard: 6822109


## Day 04

Keep the board states in a `5 x 5 x BOARD_COUNT` array. When a number `N` is drawn, replace all occurrences of `N` with `-1`.

In [17]:
using DelimitedFiles

const N = 5

day04_file = open("04.txt")
numbers = parse.(Int, split(readline(day04_file), ','))
boards = permutedims(
    reshape(readdlm(day04_file, Int), N, :, N),
    (1, 3, 2),
)

function wins(board_idx)
    any(
        all(boards[:, idx, board_idx] .< 0) || all(boards[idx, :, board_idx] .< 0)
        for idx in 1:N
    )
end

function score(board_idx, drawn_number)
    drawn_number * sum(filter(>=(0), boards[:, :, board_idx]))
end

live_boards = Set(1:size(boards, 3))
easy_answer, hard_answer = nothing, nothing
for num in numbers
    boards[boards .== num] .= -1
    winning_boards = Set(idx for idx in live_boards if wins(idx))
    if length(winning_boards) == 1
        current_score = score(first(winning_boards), num)
        if isnothing(easy_answer)
            easy_answer = current_score
        end
        hard_answer = current_score
    end
    setdiff!(live_boards, winning_boards)
end

println("easy: ", easy_answer)
println("hard: ", hard_answer)

easy: 44088
hard: 23670


## Day 05

In [75]:
segments = open("05.txt") do f
    map(readlines(f)) do line
        split(line, " -> ") .|> x -> split(x, ",") .|> x -> parse(Int, x)
    end
end

function count_overlaps(include_diagonals)
    counts = Dict()
    
    function add_point(x, y)
        counts[(x, y)] = get!(counts, (x, y), 0) + 1
    end
    
    for ((x1, y1), (x2, y2)) in segments
        Δx, Δy = sign(x2 - x1), sign(y2 - y1)
        if include_diagonals || (Δx == 0 || Δy == 0)
            while x1 != x2 || y1 != y2
                add_point(x1, y1)
                x1 += Δx
                y1 += Δy
            end
            add_point(x1, y1)
        end
    end
    
    count(>(1), values(counts))
end


println("easy: ", count_overlaps(false))
println("hard: ", count_overlaps(true))

easy: 5585
hard: 17193


## Day 06

In [112]:
set_count(counts, idx, val) = counts[idx + 1] = val
get_count(counts, idx) = counts[idx + 1]
inc_count(counts, idx, by) = set_count(counts, idx, by + get_count(counts, idx))

function initial_counts()
    res = zeros(Int, 9)
    for f in parse.(Int, split(readline("06.txt"), ','))
        inc_count(res, f, 1)
    end
    res
end

function step(counts)
    res = [
        get_count(counts, mod(idx + 1, 9))
        for idx in 0:8
    ]
    inc_count(res, 6, get_count(counts, 0))
    res
end

function iterate_steps(n)
    res = initial_counts()
    for _ in 1:n
        res = step(res)
    end
    sum(res)
end

println("easy: ", iterate_steps(80))
println("hard: ", iterate_steps(256))

easy: 380758
hard: 1710623015163
